# Análisis de regresión logística

In [ ]:
install.packages(c("blorr"))

library(tidyverse)
library(blorr)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘listenv’, ‘parallelly’, ‘prodlim’, ‘future’, ‘warp’, ‘iterators’, ‘lhs’, ‘proxy’, ‘DiceDesign’, ‘patchwork’, ‘globals’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘furrr’, ‘slider’, ‘foreach’, ‘GPfit’, ‘modelenv’, ‘rootSolve’, ‘e1071’, ‘lmom’, ‘dials’, ‘hardhat’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’, ‘mvtnorm’, ‘expm’, ‘Rcpp’, ‘Exact’, ‘gld’, ‘BH’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ 

# Ejercicio

Un grupo de investigadores e investigadoras esperan que el estrés la ansiedad y la sintomatología de depresión contribuyan positivamente en los problemas de sueño de trabajadores médicos hospitalarios.

HT: El estrés, la ansiedad y depresión contribuyen a los problemas de sueño de las/los trabajadores médicos de hospital
- $H_{0}$: $β_{estres} = β_{ansiedad} = β_{depresion} = 0$
- $H_{1}$: $β_{estres} > β_{ansiedad} > β_{depresion} > 0$

Ecuación del modelo de regresión logística:

Logit (problemas de sueño) = $β_{0} - β_{estres} + β_{ansiedad} + β_{depresion}$

## Carga de datos

In [ ]:
df = haven::read_sav("https://github.com/renatoparedes/EstadisticaYPsicologiaMatematica/raw/main/INEE/Clase15_BaseRegresionLogistica.sav")
print(df, n = 10)

# A tibble: 100 × 7
   Código Sexo        Edad ProbSueño Estrés Ansiedad Depresión
   <chr>  <dbl+lbl>  <dbl> <dbl+lbl>  <dbl>    <dbl>     <dbl>
 1 001    0 [Mujer]     40 1 [Sí]         5        7         3
 2 002    1 [Hombre]    26 1 [Sí]         8        6         1
 3 003    1 [Hombre]    19 0 [No]         2        3         1
 4 004    1 [Hombre]    35 0 [No]         2        1         2
 5 005    1 [Hombre]    35 0 [No]         8        2         5
 6 006    0 [Mujer]     40 1 [Sí]        10        4         7
 7 007    0 [Mujer]     58 0 [No]         8        4         0
 8 008    0 [Mujer]     33 0 [No]         2        9         2
 9 009    1 [Hombre]    32 1 [Sí]         8        9         3
10 010    0 [Mujer]     53 1 [Sí]         8       11         6
# ℹ 90 more rows


## Análisis

In [ ]:
df %>%
  glm(ProbSueño ~ Estrés + Ansiedad + Depresión,
      family = "binomial",
      data = .) -> model1

In [ ]:
blorr::blr_model_fit_stats(model1) %>% unclass() %>% as_tibble()

loglik_null,loglik_model,m_deviance,lr_ratio,lr_pval,mcfadden,dev_df,adj_mcfadden,m_aic,cox_snell,m_bic,mckelvey,lr_df,effron,nagelkerke,count_r2,count_adj
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-68.99438,-54.71188,109.4238,28.565,2.763898e-06,0.2070096,96,0.1490339,117.4238,0.2484744,127.8444,0.3436749,3,0.2541452,0.3320105,0.72,0.3913043


In [ ]:
blorr::blr_test_hosmer_lemeshow(model1) -> hl
hl[-1] %>% as_tibble()

chisq_stat,df,pvalue
<dbl>,<dbl>,<dbl>
1.720681,8,0.9884107


In [ ]:
blorr::blr_confusion_matrix(model1, cutoff = 0.5) -> mc
mc$conf_matrix %>% as_tibble()

Prediction,Reference,n
<chr>,<chr>,<int>
0,0,42
1,0,12
0,1,16
1,1,30


In [ ]:
mc[-3] %>% as_tibble() %>% select(accuracy, sensitivity, specificity)

accuracy,sensitivity,specificity
<dbl>,<dbl>,<dbl>
0.72,0.6521739,0.7777778


In [ ]:
model %>% broom::tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.88375669,0.70102692,-4.1136176,3.895064e-05
Estrés,0.18067643,0.11390465,1.5862076,1.126922e-01
Ansiedad,0.22914056,0.09764158,2.3467519,1.893786e-02
Depresión,0.08139973,0.09883809,0.8235664,4.101860e-01


In [ ]:
model %>% broom::tidy(exponentiate = T, conf.int = TRUE)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.05592428,0.70102692,-4.1136176,3.895064e-05
Estrés,1.19802747,0.11390465,1.5862076,1.126922e-01
Ansiedad,1.25751879,0.09764158,2.3467519,1.893786e-02
Depresión,1.08480444,0.09883809,0.8235664,4.101860e-01
